In [1]:
import pickle

heterodata = pickle.load(open("hetero_graph_final.pkl", "rb"))

/home/amosd/miniconda3/envs/gnn/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
heterodata

HeteroData(
  paper={
    license=[2381173],
    doi=[2381173],
    pages=[2381173],
    journal=[2381173],
    date=[2381173],
    id=[2381173],
    name=[2381173],
    num_nodes=2381173,
  },
  author={
    name=[1894426],
    num_nodes=1894426,
  },
  category={
    name=[176],
    num_nodes=176,
  },
  word={
    name=[2465063],
    num_nodes=2465063,
  },
  journal={
    name=[815904],
    num_nodes=815904,
  },
  (paper, written_by, author)={ edge_index=[2, 10595660] },
  (paper, has_category, category)={ edge_index=[2, 4035894] },
  (paper, has_word, word)={
    edge_index=[2, 217335585],
    edge_attr=[217335585],
  },
  (paper, has_titleword, word)={ edge_index=[2, 18672617] },
  (paper, in_journal, journal)={ edge_index=[2, 2381173] },
  (word, co_occurs_with, word)={
    edge_index=[2, 160852251],
    edge_attr=[160852251],
  }
)

In [3]:
heterodata['paper']['license'] = [license if license is not None else 'None' for license in heterodata['paper']['license']]

In [4]:
# idx = 336
# idx2 = 4
# a= heterodata[heterodata.edge_types[idx2]].edge_index
# from_node = heterodata[heterodata.edge_types[idx2][0]].name[a[0,idx]]
# to_node = heterodata[heterodata.edge_types[idx2][2]].name[a[1,idx]]

# print(heterodata.edge_types[idx2][0], from_node)
# print(heterodata.edge_types[idx2][2], to_node)

In [5]:
# heterodata[('paper', 'in_journal', 'journal')] = heterodata[('paper', 'in_journal', 'journal-ref')]
# del heterodata[('paper', 'in_journal', 'journal-ref')]

# remove isolated nodes
import torch_geometric.transforms as T
transform = T.Compose([
       #T.RemoveIsolatedNodes(),
       T.RemoveDuplicatedEdges(),
       # T.ToUndirected(merge=False) # don't merge reversed edges into the original edge type
])
from torch_geometric.data import HeteroData 

# from copy import deepcopy
# data = deepcopy(heterodata)
# for node_type in heterodata.node_types[-5:]:
       # heterodata[node_type].num_nodes = hetero

# for edge_type in  heterodata.edge_types[-:]:
       # print(edge_type)
       # 
       # del data[edge_type]
       # del data[list(reversed(heterodata.edge_types))[5]]

# for node_type in heterodata.node_types[:6]:
#        print(node_type)
#        del data[node_type]
# test = transform(data)
heterodata = transform(heterodata)

In [6]:
import torch
# id mapping so we can remove the attributes and then remove isolated nodes, later we can add the attributes back
id_dict = {}
more_mappings = {}
heterodata_dict = heterodata.to_dict()
for nodetype in heterodata.node_types:
    print(nodetype)
    ids = torch.arange(heterodata[nodetype].num_nodes) 
    id_mapping = {i.item():name for i, name in zip(ids, heterodata[nodetype].name)}
    if nodetype =='paper':
        for key in ['license','doi','pages','journal','date','id']:
            more_mappings[key] = {i.item():name for i, name in zip(ids, heterodata[nodetype][key])}
    
    heterodata[nodetype].x = ids.squeeze(-1)
    id_dict[nodetype] = id_mapping
    
    for key in heterodata_dict[nodetype].keys():
        if key != 'x':
            print('del', nodetype, key)
            del heterodata[nodetype][key]

paper
del paper license
del paper doi
del paper pages
del paper journal
del paper date
del paper id
del paper name
del paper num_nodes
author
del author name
del author num_nodes
category
del category name
del category num_nodes
word
del word name
del word num_nodes
journal
del journal name
del journal num_nodes


In [7]:
heterodata

HeteroData(
  paper={ x=[2381173] },
  author={ x=[1894426] },
  category={ x=[176] },
  word={ x=[2465063] },
  journal={ x=[815904] },
  (paper, written_by, author)={ edge_index=[2, 10586478] },
  (paper, has_category, category)={ edge_index=[2, 4035894] },
  (paper, has_word, word)={
    edge_index=[2, 156112429],
    edge_attr=[156112429],
  },
  (paper, has_titleword, word)={ edge_index=[2, 18361007] },
  (paper, in_journal, journal)={ edge_index=[2, 2381173] },
  (word, co_occurs_with, word)={
    edge_index=[2, 160852251],
    edge_attr=[160852251],
  }
)

In [8]:
# check correct edge types:
transform = T.Compose([
       T.RemoveIsolatedNodes(),
       #    T.RemoveDuplicatedEdges(),
       # T.ToUndirected(merge=False) # don't merge reversed edges into the original edge type
])

heterodata = transform(heterodata)

In [9]:
heterodata


HeteroData(
  paper={ x=[2381173] },
  author={ x=[1894426] },
  category={ x=[176] },
  word={ x=[2464870] },
  journal={ x=[815904] },
  (paper, written_by, author)={ edge_index=[2, 10586478] },
  (paper, has_category, category)={ edge_index=[2, 4035894] },
  (paper, has_word, word)={
    edge_index=[2, 156112429],
    edge_attr=[156112429],
  },
  (paper, has_titleword, word)={ edge_index=[2, 18361007] },
  (paper, in_journal, journal)={ edge_index=[2, 2381173] },
  (word, co_occurs_with, word)={
    edge_index=[2, 160852251],
    edge_attr=[160852251],
  }
)

In [10]:
# from tqdm.auto import tqdm
# import json 
# import pandas as pd 

# def read_first_n_lines(file_path, n=10000):
#     data = []
#     with open(file_path, 'r') as file:
#         for i, line in tqdm(enumerate(file), total=n):
#             if i >= n:
#                 break
#             try:
#                 data.append(json.loads(line))
#             except json.JSONDecodeError:
#                 continue
#     return pd.DataFrame(data)

# file_path = '../make_the_graph/data/arxiv-metadata-oai-snapshot.json'

# # DataFrame erstellen
# df = read_first_n_lines(file_path, 1000)

In [11]:
heterodata = T.ToUndirected()(heterodata)


In [12]:
# map back
for nodetype in heterodata.node_types:
    print(nodetype)
    id_mapping = id_dict[nodetype]
    heterodata[nodetype].name = [id_mapping[i.item()] for i in heterodata[nodetype].x]
    
    heterodata[nodetype].num_nodes = len(heterodata[nodetype].name)
    
    if nodetype =='paper':
        for key in ['license','doi','pages','journal','date','id']:
            heterodata[nodetype][key] = [more_mappings[key][i.item()] for i in heterodata[nodetype].x]
    
    del heterodata[nodetype].x

paper
author
category
word
journal


In [13]:
# idx = 330
# idx2 = 4
# a= heterodata[heterodata.edge_types[idx2]].edge_index
# from_node = heterodata[heterodata.edge_types[idx2][0]].name[a[0,idx]]
# to_node = heterodata[heterodata.edge_types[idx2][2]].name[a[1,idx]]

# print(heterodata.edge_types[idx2][0], from_node)
# print(heterodata.edge_types[idx2][2], to_node)

In [14]:
import pickle
import os
if not os.path.exists('arxiv_author_paper_graph_no_features_bare.pkl'):
    pickle.dump(heterodata, open("arxiv_author_paper_graph_no_features_bare.pkl", "wb"))

In [15]:
# paper nans to median
import math
non_nan = [num for num in heterodata['paper'].pages if not math.isnan(num)]
median = non_nan[len(non_nan)//2]
heterodata['paper'].pages = [median if math.isnan(num) else num for num in heterodata['paper'].pages]


In [16]:
# categorical 
unique_licenses= list(set(heterodata['paper'].license))
categories = {unique_licenses[i]:i for i in range(len(unique_licenses))}
heterodata['paper'].license = [categories[license] for license in heterodata['paper'].license]
import torch 
# onehot
onehot = torch.nn.functional.one_hot(torch.tensor(heterodata['paper'].license ))
heterodata['paper'].license = onehot

In [17]:
given_timestamp = heterodata['paper'].date[0]
import datetime
start_timestamp_epoch = datetime.datetime(1970, 1, 1, 0, 0, 0, tzinfo=given_timestamp.tzinfo)

# Calculate the difference in hours divided by 
difference_in_hours_by_4 = (given_timestamp - start_timestamp_epoch).total_seconds() // (3600*4) 

In [18]:
heterodata['paper'].date = [((timestamp - start_timestamp_epoch).total_seconds() // (3600*4)) for timestamp in heterodata['paper'].date]

In [19]:
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('all-MiniLM-L6-v2')
# skill_sbert_embeddings = embedder.encode(skill_nodes['skill'].tolist(), convert_to_numpy=True)
from tqdm.auto import tqdm
# for each node type create the x attribute from the embeddings of the node names
for node_type in heterodata.node_types:
    print(node_type)
    # do it in batches
    x = torch.zeros(heterodata[node_type].num_nodes, 384).float()
    batch_size = 10000
    for i in tqdm(range(0, heterodata[node_type].num_nodes, 10000)):
        x[i:i+10000] = torch.tensor(embedder.encode(heterodata[node_type].name[i:i+10000], convert_to_numpy=True))
        
        
    heterodata[node_type].x = x
    # heterodata[node_type].x = embedder.encode(heterodata[node_type].name, convert_to_numpy=False)

paper


100%|██████████| 239/239 [1:01:52<00:00, 15.54s/it]


author


100%|██████████| 190/190 [28:53<00:00,  9.12s/it]


category


100%|██████████| 1/1 [00:00<00:00,  6.71it/s]


word


 63%|██████▎   | 156/247 [26:09<16:07, 10.64s/it]

In [ ]:
heterodata['paper'].x = torch.concatenate((heterodata['paper'].x,heterodata['paper'].license), dim=1)
heterodata['paper'].x  = torch.concatenate((heterodata['paper'].x, torch.tensor(heterodata['paper'].date).unsqueeze(1)), dim=1)

In [ ]:
for node_type in heterodata.node_types:
    del heterodata[node_type].name 
    if node_type == 'paper':
        for key in ['license','doi','pages','journal','date','id']:
            del heterodata[node_type][key]

In [ ]:
# save 
import pickle
import os
if not os.path.exists('arxiv_author_paper_graph_training.pkl'):
    pickle.dump(heterodata, open("arxiv_author_paper_graph_training.pkl", "wb"))

In [ ]:
# for nodetype in heterodata.node_types:
#     print(nodetype, heterodata[nodetype].x.shape, heterodata[nodetype].num_nodes)

In [ ]:
def describe(heterodata):
    heterodata_dict = heterodata.to_dict()
    # print('has isolated nodes:', heterodata.has_isolated_nodes())
    # print('has self loops:', heterodata.has_self_loops())
    # print('is directed', heterodata.is_directed())
    for nodetype in heterodata.node_types:
        print(nodetype)
        print('  ',heterodata[nodetype].num_nodes)
        print('  ',heterodata_dict[nodetype].keys())
        
    for edgetype in heterodata.edge_types:
        print(edgetype)
        print('  ',heterodata[edgetype].num_edges)
        print('  ',heterodata[edgetype])

# describe(heterodata)

paper
   None
   dict_keys(['name'])
author
   None
   dict_keys(['name'])
category
   None
   dict_keys(['name'])
word
   None
   dict_keys(['name'])
journal
   None
   dict_keys(['name'])
('paper', 'written_by', 'author')
   3067
   {'edge_index': tensor([[   0,    0,    0,  ...,  997,  998,  999],
        [   0,    1,    2,  ..., 2883, 2884, 2885]])}
('paper', 'has_category', 'category')
   1502
   {'edge_index': tensor([[  0,   1,   1,  ..., 997, 998, 999],
        [  0,   1,   2,  ...,  10,  52,  15]])}
('paper', 'has_word', 'word')
   57073
   {'edge_index': tensor([[    0,     0,     0,  ...,   999,   999,   999],
        [    0,     1,     2,  ..., 10209, 10210, 10211]]), 'edge_attr': tensor([0.0942, 0.0789, 0.2729,  ..., 0.1235, 0.1235, 0.1235])}
('paper', 'has_titleword', 'word')
   7354
   {'edge_index': tensor([[    0,     0,     0,  ...,   999,   999,   999],
        [    2,     7,    41,  ..., 10191, 10192, 10193]])}
('paper', 'in_journal', 'journal')
   1000
   {'edge_in

/home/amosd/miniconda3/envs/gnn/lib/python3.11/site-packages/torch_geometric/data/storage.py:327: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'name'}'. Please explicitly set 'num_nodes' as an attribute of 'data[paper]' to suppress this warning
  warnings.warn(
/home/amosd/miniconda3/envs/gnn/lib/python3.11/site-packages/torch_geometric/data/storage.py:327: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'name'}'. Please explicitly set 'num_nodes' as an attribute of 'data[author]' to suppress this warning
  warnings.warn(
/home/amosd/miniconda3/envs/gnn/lib/python3.11/site-packages/torch_geometric/data/storage.py:327: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'name'}'. Please explicitly set 'num_nodes' as an attribute of 'data[category]' to suppress this warning
  warnings.warn(
/home/amosd/miniconda3/envs/gnn/lib/python3.11/site-packages/torch_geometric/data/storage.py:327: UserWarning: Una